In [37]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import seaborn as sns

In [2]:
# pull starts raw data
starts_df = pd.read_csv('../capstone_data/skillshare_2022_starts.csv')

/Users/travisstowe/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (24,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# filter starts_df to our needed peramenters.
# only annual
starts_df = starts_df[starts_df['plan_length'] == 12]
# not B2B
starts_df = starts_df[starts_df['is_team'] == False]
# no scholarships
starts_df = starts_df[starts_df['is_scholarship'] == False]
# has a free trial
starts_df = starts_df[starts_df['is_direct_to_paid'] == False]
starts_df = starts_df[starts_df['had_trial'] == True]
# remove updates; too complicated
starts_df = starts_df[starts_df['was_upgraded'] == False]
# no special trial lengths
starts_df = starts_df[starts_df['trial_length_offer'].isin(['One Week', 'One Month'])]

In [4]:
# for some reason user_uid is a float only on this dataframe. Change it to an int.
starts_df['user_uid'] = starts_df['user_uid'].astype(int)

# make a day version of the trial start and end dates.
starts_df['trial_end_day'] = pd.to_datetime(starts_df.original_trial_end)
starts_df['trial_start_day'] = pd.to_datetime(starts_df.create_time)
starts_df['cancellation_day'] = pd.to_datetime(starts_df.cancellation_time)
starts_df['first_payment_day'] = pd.to_datetime(starts_df.first_payment_time)

# Need to breakdown if the cancellation came before or after the first payment.
starts_df['is_cancel_during_trial'] = 0
starts_df['is_cancel_during_trial'][starts_df['cancellation_day'] <= starts_df['trial_end_day']] = 1

In [5]:
# these are the columns have some potential value to prediction.
prediction_cols = ['user_uid', 'create_time', 'first_payment_time', 'last_payment_attempt', 
                   'is_cancel_during_trial', 'trial_end_day', 'trial_start_day',
                   'last_failed_payment_attempt', 'user_cancellation_time', 'cancellation_time', 
                   'refund_time', 'coupon_id', 'coupon_trial_length', 'payment_provider', 'payment_ux', 
                   'is_refunded', 'is_cancelled', 'has_paid', 'trial_end', 'first_payment_currency_code', 'original_trial_end', 
                   'extended_trial_end', 'was_trial_extended', 'is_trial_extension', 'is_split_trial', 
                   'trial_length_days', 'trial_length_offer', 'sub_utm_source', 'sub_utm_campaign', 
                   'sub_utm_medium', 'sub_utm_term', 'sub_utm_channel', 'referral_source', 'eligible_trial_number']

clean_df = starts_df[prediction_cols]

In [6]:
# add a successful conversion column.
clean_df['success'] = 0

# set to 1 if they paid
clean_df['success'][clean_df['first_payment_time'].notnull()] = 1

# return to 0 if they got a refund.
clean_df['success'][clean_df['is_refunded']==1] = 0

In [7]:
# create category code columns for each of the most relevant predictive columns
clean_df['payment_provider_cat'] = clean_df['payment_provider'].astype('category')
clean_df['payment_provider_cat_codes'] = clean_df['payment_provider_cat'].cat.codes

clean_df['payment_ux_cat'] = clean_df['payment_ux'].astype('category')
clean_df['payment_ux_cat_codes'] = clean_df['payment_ux_cat'].cat.codes

clean_df['trial_length_offer_cat'] = clean_df['trial_length_offer'].astype('category')
clean_df['trial_length_offer_cat_codes'] = clean_df['trial_length_offer_cat'].cat.codes

clean_df['sub_utm_channel_cat'] = clean_df['sub_utm_channel'].astype('category')
clean_df['sub_utm_channel_cat_codes'] = clean_df['sub_utm_channel_cat'].cat.codes

clean_df['sub_utm_source_cat'] = clean_df['sub_utm_source'].astype('category')
clean_df['sub_utm_source_cat_codes'] = clean_df['sub_utm_source_cat'].cat.codes

clean_df['user_uid'] = clean_df['user_uid'].astype(int)

In [8]:
# export all of the lookup columns for EDA analysis.
payment_provider_lookup_df = clean_df.groupby(
    by=['payment_provider', 'payment_provider_cat_codes']).agg(
        volume=pd.NamedAgg(column='user_uid', aggfunc='count')).reset_index()
payment_provider_lookup_df.to_csv('lookup_payment_providers.csv')

payment_ux_df = clean_df.groupby(
    by=['payment_ux', 'payment_ux_cat_codes']).agg(
        volume=pd.NamedAgg(column='user_uid', aggfunc='count')).reset_index()
payment_ux_df.to_csv('lookup_payment_ux.csv')

trial_length_df = clean_df.groupby(
    by=['trial_length_offer', 'trial_length_offer_cat_codes']).agg(
        volume=pd.NamedAgg(column='user_uid', aggfunc='count')).reset_index()
trial_length_df.to_csv('lookup_trial_length_offer.csv')

sub_utm_channel_df = clean_df.groupby(
    by=['sub_utm_channel', 'sub_utm_channel_cat_codes']).agg(
        volume=pd.NamedAgg(column='user_uid', aggfunc='count')).reset_index()
sub_utm_channel_df.to_csv('lookup_sub_utm_channel.csv')

sub_utm_source_df = clean_df.groupby(
    by=['sub_utm_source', 'sub_utm_source_cat_codes']).agg(
        volume=pd.NamedAgg(column='user_uid', aggfunc='count')).reset_index()
sub_utm_source_df.to_csv('lookup_sub_utm_source.csv')

In [9]:
# make a df of only the columms we want for prediction
p_cols = ['user_uid', 'create_time', 'success', 'cancellation_time', 'payment_provider_cat_codes', 
          'payment_ux_cat_codes', 'trial_length_offer_cat_codes', 'sub_utm_channel_cat_codes', 
          'sub_utm_source_cat_codes', 'trial_start_day', 'trial_end_day', 'is_cancel_during_trial']
mlready_df = clean_df[p_cols]

# remove duplicates by keeping the most recent subscription for any user_uid.
mlready_df.sort_values(by='create_time', inplace=True)
mlready_df = mlready_df.drop_duplicates(subset=['user_uid'], keep='last')

In [10]:
# append the video views data onto the starts.
# creation of video views data file found in data_combine_v1.py
vviews_df = pd.read_csv('../capstone_data/skillshare_2022_all_views.csv')

# rename uid and remove unneeded columns.
vviews_df.rename(columns={'uid':'user_uid'}, inplace=True)
del vviews_df['Unnamed: 0']

In [11]:
# merge vviews to starts data and fill in missing data with 0.
combo_df = mlready_df.merge(vviews_df, how='left', on='user_uid')
combo_df = combo_df.fillna(0.0)

In [12]:
#### begin to translate non-video activity to merge.

# We need to make sure non-video activity is during the trial for the user
# so we need a dataframe of the the trial date range for the user.
trial_ends = starts_df[['user_uid', 'trial_start_day', 'trial_end_day']]

In [13]:
### Merge on Comments Data.
# pull the comments data.
comments_df = pd.read_csv('../capstone_data/skillshare_2022_comments.csv')

# change user_id column name for easy merge.
comments_df.rename(columns={'user_id':'user_uid'}, inplace=True)

# merge on the trial start and end.
comments_df = comments_df.merge(trial_ends, how='left', on='user_uid')

# round create_time to created_day.
comments_df['create_day'] = pd.to_datetime(comments_df.create_time).dt.date

# filter data to comments that happened during the user's trial
comments_df = comments_df[comments_df['create_day'] > comments_df['trial_start_day']]
comments_df = comments_df[comments_df['create_day'] < comments_df['trial_end_day']]

# make a groupby that for each user_uid that includes num of comments and total comment score.
comment_gb_df = comments_df.groupby(by=['user_uid']).agg(
    comment_volume=pd.NamedAgg(column='id', aggfunc='count'), 
    comment_score=pd.NamedAgg(column='score', aggfunc='sum')).reset_index()

# merge onto main dataframe.
combo_df = combo_df.merge(comment_gb_df, how='left', on='user_uid')

In [14]:
### Merge on Discussions Data.
# Follow the same process as comments for discussions.
discussions_df = pd.read_csv('../capstone_data/skillshare_2022_discussions.csv')
discussions_df.rename(columns={'user_id':'user_uid'}, inplace=True)
discussions_df = discussions_df.merge(trial_ends, how='left', on='user_uid')
discussions_df['create_day'] = pd.to_datetime(discussions_df.create_time)
discussions_df = discussions_df[discussions_df['create_day'] > discussions_df['trial_start_day']]
discussions_df = discussions_df[discussions_df['create_day'] < discussions_df['trial_end_day']]
discussions_gb_df = discussions_df.groupby(by=['user_uid']).agg(
    discussion_volume=pd.NamedAgg(column='id', aggfunc='count'), 
    discussion_score=pd.NamedAgg(column='score', aggfunc='sum')).reset_index()
combo_df = combo_df.merge(discussions_gb_df, how='left', on='user_uid')

In [15]:
### Merge on Follows Data.
# Follow the same process as comments for follows.
follows_df = pd.read_csv('../capstone_data/skillshare_2022_follows.csv')
follows_df.rename(columns={'follower_uid':'user_uid'}, inplace=True)
follows_df = follows_df.merge(trial_ends, how='left', on='user_uid')
follows_df['follow_day'] = pd.to_datetime(follows_df.follow_time)
follows_df = follows_df[follows_df['follow_day'] > follows_df['trial_start_day']]
follows_df = follows_df[follows_df['follow_day'] < follows_df['trial_end_day']]

# make a groupby by user_uid that counts the number of follows.
follows_gb_df = follows_df.groupby(by=['user_uid']).agg(
    follow_volume=pd.NamedAgg(column='target_uid', aggfunc='count')).reset_index()
combo_df = combo_df.merge(follows_gb_df, how='left', on='user_uid')

In [16]:
### Merge on Projects Data.
# Follow the same process as comments for follows.
projects_df = pd.read_csv('../capstone_data/skillshare_2022_projects.csv')
projects_df.rename(columns={'uid':'user_uid'}, inplace=True)
projects_df = projects_df.merge(trial_ends, how='left', on='user_uid')
projects_df['create_day'] = pd.to_datetime(projects_df.create_time)
projects_df = projects_df[projects_df['create_day'] > projects_df['trial_start_day']]
projects_df = projects_df[projects_df['create_day'] < projects_df['trial_end_day']]
projects_gb_df = projects_df.groupby(by=['user_uid']).agg(
    projects_volume=pd.NamedAgg(column='id', aggfunc='count'), 
    projects_score=pd.NamedAgg(column='num_up', aggfunc='sum')).reset_index()
combo_df = combo_df.merge(projects_gb_df, how='left', on='user_uid')

In [17]:
### Merge on Reviews Data.
# Follow the same process as comments for follows.
reviews_df = pd.read_csv('../capstone_data/skillshare_2022_reviews.csv')
reviews_df.rename(columns={'uid':'user_uid'}, inplace=True)
reviews_df = reviews_df.merge(trial_ends, how='left', on='user_uid')
reviews_df['create_day'] = pd.to_datetime(reviews_df.create_time)
reviews_df = reviews_df[reviews_df['create_day'] > reviews_df['trial_start_day']]
reviews_df = reviews_df[reviews_df['create_day'] < reviews_df['trial_end_day']]

# make a groupby for each user and their volume of reviews and avg review score.
reviews_gb_df = reviews_df.groupby(by=['user_uid']).agg(
    review_volume=pd.NamedAgg(column='review_id', aggfunc='count'), 
    rating_avg=pd.NamedAgg(column='rating', aggfunc='mean')).reset_index()
combo_df = combo_df.merge(reviews_gb_df, how='left', on='user_uid')

In [18]:
# export data
combo_df.to_csv('skillshare_combined.csv', index=False)

In [19]:
output_dict = {'total starts': len(combo_df),
              'overall_p1nr': combo_df.success.sum() / len(combo_df)}

In [20]:
cancelled_df = combo_df[combo_df['is_cancel_during_trial']==1]

In [21]:
output_dict['cancelled'] = len(cancelled_df)
output_dict['cancelled_p1nr'] = cancelled_df.success.sum() / len(cancelled_df)

In [22]:
output_dict

{'total starts': 473932,
 'overall_p1nr': 0.2396546339981263,
 'cancelled': 195213,
 'cancelled_p1nr': 0.01041426544338748}

In [23]:
not_cancelled_df = combo_df[combo_df['is_cancel_during_trial']==0]
output_dict['not_cancelled'] = len(not_cancelled_df)
output_dict['not_cancelled_p1nr'] = not_cancelled_df.success.sum() / len(not_cancelled_df)

In [24]:
output_dict

{'total starts': 473932,
 'overall_p1nr': 0.2396546339981263,
 'cancelled': 195213,
 'cancelled_p1nr': 0.01041426544338748,
 'not_cancelled': 278719,
 'not_cancelled_p1nr': 0.4002131178714045}

In [26]:
not_cancelled_df = not_cancelled_df.fillna(0.0)
not_cancelled_df['nonV_engagement'] = not_cancelled_df['comment_volume'] + not_cancelled_df['discussion_volume'] + not_cancelled_df['follow_volume'] + not_cancelled_df['projects_volume'] + not_cancelled_df['review_volume']


In [27]:
not_cancelled_df['has_nonV_engagement'] = 0
not_cancelled_df['has_nonV_engagement'][not_cancelled_df['nonV_engagement'] > 0] = 1

In [28]:
not_cancelled_nonve_df = not_cancelled_df[not_cancelled_df['has_nonV_engagement']==0]
output_dict['no_nonV'] = len(not_cancelled_nonve_df)
output_dict['no_nonV_p1nr'] = not_cancelled_nonve_df.success.sum() / len(not_cancelled_nonve_df)

In [29]:
output_dict

{'total starts': 473932,
 'overall_p1nr': 0.2396546339981263,
 'cancelled': 195213,
 'cancelled_p1nr': 0.01041426544338748,
 'not_cancelled': 278719,
 'not_cancelled_p1nr': 0.4002131178714045,
 'no_nonV': 217346,
 'no_nonV_p1nr': 0.3902303239995215}

In [30]:
not_cancelled_nve_df = not_cancelled_df[not_cancelled_df['has_nonV_engagement']==1]
output_dict['nonV'] = len(not_cancelled_nve_df)
output_dict['nonV_p1nr'] = not_cancelled_nve_df.success.sum() / len(not_cancelled_nve_df)

In [31]:
output_dict

{'total starts': 473932,
 'overall_p1nr': 0.2396546339981263,
 'cancelled': 195213,
 'cancelled_p1nr': 0.01041426544338748,
 'not_cancelled': 278719,
 'not_cancelled_p1nr': 0.4002131178714045,
 'no_nonV': 217346,
 'no_nonV_p1nr': 0.3902303239995215,
 'nonV': 61373,
 'nonV_p1nr': 0.4355661284278103}

In [34]:
not_cancelled_df['totalmin-day-3'] = not_cancelled_df['day-1'] + not_cancelled_df['day-2'] + not_cancelled_df['day-3']
not_cancelled_df['totalmin-day-7'] = not_cancelled_df['totalmin-day-3'] + not_cancelled_df['day-4'] + not_cancelled_df['day-5'] + not_cancelled_df['day-6'] + not_cancelled_df['day-7']
not_cancelled_df['totalmin-day-14'] = not_cancelled_df['totalmin-day-7'] + not_cancelled_df['day-8'] + not_cancelled_df['day-9'] + not_cancelled_df['day-10'] + not_cancelled_df['day-11'] +  + not_cancelled_df['day-12'] + not_cancelled_df['day-13'] + not_cancelled_df['day-14']
not_cancelled_df['totalmin'] = not_cancelled_df['totalmin-day-14']
for x in range(15, 31):
    daystr = 'day-' + str(x)
    not_cancelled_df['totalmin'] = not_cancelled_df['totalmin'] + not_cancelled_df[daystr]

In [50]:
for a in range(31):
    x = a * 10.0
    print('Checking:', x)
    check_bad = not_cancelled_df[not_cancelled_df['totalmin-day-14']<=x]
    check_good = not_cancelled_df[not_cancelled_df['totalmin-day-14']>x]
    print('Below:', "{0:0.3f}".format(check_bad.success.sum() / len(check_bad)), '     Above:', "{0:0.3f}".format(check_good.success.sum() / len(check_good)))
    print('----------------------------')
    print(' ')

Checking: 0.0
Below: 0.302      Above: 0.426
----------------------------
 
Checking: 10.0
Below: 0.312      Above: 0.427
----------------------------
 
Checking: 20.0
Below: 0.314      Above: 0.429
----------------------------
 
Checking: 30.0
Below: 0.316      Above: 0.430
----------------------------
 
Checking: 40.0
Below: 0.317      Above: 0.431
----------------------------
 
Checking: 50.0
Below: 0.318      Above: 0.432
----------------------------
 
Checking: 60.0
Below: 0.320      Above: 0.432
----------------------------
 
Checking: 70.0
Below: 0.321      Above: 0.433
----------------------------
 
Checking: 80.0
Below: 0.321      Above: 0.434
----------------------------
 
Checking: 90.0
Below: 0.322      Above: 0.434
----------------------------
 
Checking: 100.0
Below: 0.323      Above: 0.435
----------------------------
 
Checking: 110.0
Below: 0.324      Above: 0.435
----------------------------
 
Checking: 120.0
Below: 0.325      Above: 0.435
----------------------------

In [52]:
not_cancelled_nve_df = check_good[check_good['has_nonV_engagement']==1]
not_cancelled_nve_df.success.sum() / len(not_cancelled_nve_df)


0.4441627204593758

In [ ]:
# not_cancelled_df['mw_in_3'] = not_cancelled_df['day-1']
# not_cancelled_df['mw_in_3_above_15'] = 0
# not_cancelled_df['mw_in_3_above_15'][not_cancelled_df['mw_in_3'] > 3600] = 1
# above_15 = not_cancelled_df[not_cancelled_df['mw_in_3_above_15'] == 1]
# below_15 = not_cancelled_df[not_cancelled_df['mw_in_3_above_15'] == 0]
# print('above 15 Rate:', len(above_15[above_15['success']==1]) / len(above_15))
# print('below 15 Rate:', len(below_15[below_15['success']==1]) / len(below_15))